# Practical 04 - RRT Algorithm

In [41]:
from Practical04_Support.Obstacle import *
from Practical04_Support.path_animation import *
import meshcat.geometry as g
import meshcat.transformations as tf
import os

from ece4078.Utility import StartMeshcat

import numpy as np
import random

# Import dependencies and set random seed
seed_value = 5
# 1. Set `PYTHONHASHSEED` environment variable at a fixed value
os.environ['PYTHONHASHSEED'] = str(seed_value)
# 2. Set `python` built-in pseudo-random generator at a fixed value
random.seed(seed_value)
# 3. Set `numpy` pseudo-random generator at a fixed value
np.random.seed(seed_value)

In [42]:
vis = StartMeshcat()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7019/static/


The class below shows a possible implementation of the RRT planning algorithm. The main methods are:

``get_random_node(.)``, this method uniformly samples a new point in the space.

``get_nearest_node_index(.)``, this method finds the index of the node in the tree that is the closest to a query node.

``steer(.)``, given two nodes n_expansion and n_random to_node, this method generate a node n_nearby such that n_nearby is “closer” to n_random than to n_expansion.

``is_collision_free(.)``, this method determines whether a new node is within the free-collision space

``planning(.)``, this is the main algorithm. Starting with a node at the start location, this method incrementally adds new nodes to the tree roadmap. The main steps are:

1. Generate a random node ``rnd_node``

2. Select the node in the tree that is the closest ``rnd_node``. We refer to this node as ``expansion_node``.

3. Choose a collision-free node along the path between ``expansion_node`` and ``rnd_node``. Add the new node to the tree roadmap. This is done by using the ``steer(.)`` and ``is_collision_free(.)`` methods one after the other.

4. Repeat steps 1 to 3 until the maximum number of nodes has been reached or a path between start and goal nodes has been found.

**TODO**:
- Complete the ``planning(.)`` method. 


In [91]:
def read_true_map(fname):
    """Read the ground truth map and output the pose of the ArUco markers and 5 target fruits&vegs to search for

    @param fname: filename of the map
    @return:
        1) list of targets, e.g. ['lemon', 'tomato', 'garlic']
        2) locations of the targets, [[x1, y1], ..... [xn, yn]]
        3) locations of ArUco markers in order, i.e. pos[9, :] = position of the aruco10_0 marker
    """
    with open(fname, 'r') as fd:
        gt_dict = json.load(fd)
        fruit_list = []
        fruit_true_pos = []
        aruco_true_pos = np.empty([10, 2])

        # remove unique id of targets of the same type
        for key in gt_dict:
            x = np.round(gt_dict[key]['x'], 1)
            y = np.round(gt_dict[key]['y'], 1)

            if key.startswith('aruco'):
                if key.startswith('aruco10'):
                    aruco_true_pos[9][0] = x
                    aruco_true_pos[9][1] = y
                else:
                    marker_id = int(key[5]) - 1
                    aruco_true_pos[marker_id][0] = x
                    aruco_true_pos[marker_id][1] = y
            else:
                fruit_list.append(key[:-2])
                if len(fruit_true_pos) == 0:
                    fruit_true_pos = np.array([[x, y]])
                else:
                    fruit_true_pos = np.append(fruit_true_pos, [[x, y]], axis=0)

        return fruit_list, fruit_true_pos, aruco_true_pos

map_filename = 'M4_true_map_full.txt'
print(read_true_map(map_filename))


(['garlic', 'capsicum', 'lime', 'lemon', 'orange', 'orange', 'pumpkin', 'potato', 'potato', 'tomato'], array([[ 0.7, -0.6],
       [ 0.2, -1. ],
       [-1.1, -0.1],
       [-0.4, -1.2],
       [-0.8,  0.5],
       [ 0. ,  0.9],
       [-0.6,  1.2],
       [-0.6, -0.7],
       [ 0.3,  0.4],
       [ 0.8,  0.8]]), array([[ 1.2,  0.8],
       [ 1.1, -0.6],
       [ 0.8, -1.2],
       [-0.3, -0.7],
       [ 0. ,  1.2],
       [-0.6,  0. ],
       [-0.4,  0.8],
       [-0.8, -1.2],
       [-1.2,  0.4],
       [ 0.8,  0. ]]))


In [45]:
def read_search_list():
    """Read the search order of the target fruits

    @return: search order of the target fruits
    """
    search_list = []
    with open('search_list.txt', 'r') as fd:
        fruits = fd.readlines()

        for fruit in fruits:
            search_list.append(fruit.strip())

    return search_list
search_list = read_search_list()
print(search_list)

['garlic', 'lemon', 'lime', 'tomato', 'pumpkin']


In [82]:
# This is an adapted version of the RRT implementation done by Atsushi Sakai (@Atsushi_twi)
from Practical04_Support.Obstacle import *
from Practical04_Support.path_animation import *
import meshcat.geometry as g
import meshcat.transformations as tf
import os

from ece4078.Utility import StartMeshcat

import numpy as np
import random

# Import dependencies and set random seed
seed_value = 5
# 1. Set `PYTHONHASHSEED` environment variable at a fixed value
os.environ['PYTHONHASHSEED'] = str(seed_value)
# 2. Set `python` built-in pseudo-random generator at a fixed value
random.seed(seed_value)
# 3. Set `numpy` pseudo-random generator at a fixed value
np.random.seed(seed_value)

vis = StartMeshcat()

class RRT:
    """
    Class for RRT planning
    """

    class Node:
        """
        RRT Node
        """

        def __init__(self, x, y):
            self.x = x
            self.y = y
            self.path_x = []
            self.path_y = []
            self.parent = None

    def __init__(self, start,
                 goal,
                 obstacle_list,
                 width,
                 height,
                 expand_dis, 
                 path_resolution, 
                 max_points):
        """
        Setting Parameter
        start:Start Position [x,y]
        goal:Goal Position [x,y]
        obstacle_list: list of obstacle objects
        width, height: search area
        expand_dis: min distance between random node and closest node in rrt to it
        path_resolion: step size to considered when looking for node to expand
        """
        self.start = self.Node(start[0], start[1])
        self.end = self.Node(goal[0], goal[1])
        self.width = width
        self.height = height
        self.expand_dis = expand_dis
        self.path_resolution = path_resolution
        self.max_nodes = max_points
        self.obstacle_list = obstacle_list
        self.node_list = []

    def planning(self, animation=True):
        """
        rrt path planning
        animation: flag for animation on or off
        """

        self.node_list = [self.start]
        while len(self.node_list) <= self.max_nodes:
            
            # 1. Generate a random node           
            rnd_node = self.get_random_node()
            
            # 2. Find node in tree that is closest to sampled node.
            # This is the node to be expanded (q_expansion)
            expansion_ind = self.get_nearest_node_index(self.node_list, rnd_node)
            expansion_node = self.node_list[expansion_ind]

            #TODO:  Complete the last two main steps of the RRT algorithm ----------------
            # 3. Select a node (nearby_node) close to expansion_node by moving from expantion_node to rnd_node
            # Use the steer method
            nearby_node = self.steer(expansion_node, rnd_node, self.expand_dis)
            # 4. Check if nearby_node is in free space (i.e., it is collision free). If collision free, add node
            # to self.node_list
            if self.is_collision_free(nearby_node):
                self.node_list.append(nearby_node)
                if self.calc_dist_to_goal(self.node_list[-1].x, self.node_list[-1].y) <= self.expand_dis:
                    final_node = self.steer(self.node_list[-1], self.end, self.expand_dis)
                    if self.is_collision_free(final_node):
                        return self.generate_final_course(len(self.node_list) - 1)
            # Please remove return None when you start coding
            
            #ENDTODO -----------------------------------------------------------------------
                
            # If we are close to goal, stop expansion and generate path
            if self.calc_dist_to_goal(self.node_list[-1].x, self.node_list[-1].y) <= self.expand_dis:
                final_node = self.steer(self.node_list[-1], self.end, self.expand_dis)
                if self.is_collision_free(final_node):
                    return self.generate_final_course(len(self.node_list) - 1)

        return None  # cannot find path

    
    def steer(self, from_node, to_node, extend_length=float("inf")):
        """
        Given two nodes from_node, to_node, this method returns a node new_node such that new_node 
        is “closer” to to_node than from_node is.
        """
        
        new_node = self.Node(from_node.x, from_node.y)
        d, theta = self.calc_distance_and_angle(new_node, to_node)
        cos_theta, sin_theta = np.cos(theta), np.sin(theta)

        new_node.path_x = [new_node.x]
        new_node.path_y = [new_node.y]

        if extend_length > d:
            extend_length = d

        # How many intermediate positions are considered between from_node and to_node
        n_expand = math.floor(extend_length / self.path_resolution)

        # Compute all intermediate positions
        for _ in range(n_expand):
            new_node.x += self.path_resolution * cos_theta
            new_node.y += self.path_resolution * sin_theta
            new_node.path_x.append(new_node.x)
            new_node.path_y.append(new_node.y)

        d, _ = self.calc_distance_and_angle(new_node, to_node)
        if d <= self.path_resolution:
            new_node.path_x.append(to_node.x)
            new_node.path_y.append(to_node.y)

        new_node.parent = from_node

        return new_node


    def is_collision_free(self, new_node):
        """
        Determine if nearby_node (new_node) is in the collision-free space.
        """
        if new_node is None:
            return True

        points = np.vstack((new_node.path_x, new_node.path_y)).T
        for obs in self.obstacle_list:
            in_collision = obs.is_in_collision_with_points(points)
            if in_collision:
                return False
        
        return True  # safe
        
    
    def generate_final_course(self, goal_ind):
        """
        Reconstruct path from start to end node
        """
        path = [[self.end.x, self.end.y]]
        node = self.node_list[goal_ind]
        while node.parent is not None:
            path.append([node.x, node.y])
            node = node.parent
        path.append([node.x, node.y])

        return path

    def calc_dist_to_goal(self, x, y):
        dx = x - self.end.x
        dy = y - self.end.y
        return math.hypot(dx, dy)

    def get_random_node(self):
        x = self.width * np.random.random_sample()-1.5
        y = self.height * np.random.random_sample()-1.5
        rnd = self.Node(x, y)
        return rnd

    @staticmethod
    def get_nearest_node_index(node_list, rnd_node):
        # Compute Euclidean disteance between rnd_node and all nodes in tree
        # Return index of closest element
        dlist = [(node.x - rnd_node.x) ** 2 + (node.y - rnd_node.y)
                 ** 2 for node in node_list]
        minind = dlist.index(min(dlist))

        return minind

    @staticmethod
    def calc_distance_and_angle(from_node, to_node):
        dx = to_node.x - from_node.x
        dy = to_node.y - from_node.y
        d = math.hypot(dx, dy) #returns the Euclidean norm
        theta = math.atan2(dy, dx)
        return d, theta 

goal = np.array([-1.0, 1.0])
start = np.array([1.0, 0.0])


all_obstacles = [Circle(0.7, -0.1, 0.1), Circle(0.1, -0.3, 0.1), Circle(0.5, -0.7, 0.1), Circle(-0.2, 0.4, 0.1), Circle(-0.5, -0.5, 0.1), Circle(-0.8, -0.9, 0.1), Circle(-1.2, -0.2, 0.1), Circle(-0.7, 0.2, 0.1), Circle(-0.9, 0.8, 0.1), Circle(0.9, 1.0, 0.1)]
print(type([Circle(0.7, -0.1, 0.1), Circle(0.1, -0.3, 0.1), Circle(0.5, -0.7, 0.1), Circle(-0.2, 0.4, 0.1), Circle(-0.5, -0.5, 0.1), Circle(-0.8, -0.9, 0.1), Circle(-1.2, -0.2, 0.1), Circle(-0.7, 0.2, 0.1), Circle(-0.9, 0.8, 0.1), Circle(0.9, 1.0, 0.1)]))
for item in all_obstacles:
    print(type(item))
rrt = RRT(start=start, goal=goal, width=3, height=3, obstacle_list=all_obstacles,
          expand_dis=0.2, path_resolution=0.2, max_points=200)  

vis.delete()
vis.Set2DView(scale=20, center=[-1.5, 1.5, 1.5, -1.5])  # Set the center with valid coordinates
animate_path_rrt(vis, rrt)  # Remove map_width and map_height arguments
display(vis.show_inline(height=500))



You can open the visualizer by visiting the following URL:
http://127.0.0.1:7039/static/


<class 'list'>
<class 'Practical04_Support.Obstacle.Circle'>
<class 'Practical04_Support.Obstacle.Circle'>
<class 'Practical04_Support.Obstacle.Circle'>
<class 'Practical04_Support.Obstacle.Circle'>
<class 'Practical04_Support.Obstacle.Circle'>
<class 'Practical04_Support.Obstacle.Circle'>
<class 'Practical04_Support.Obstacle.Circle'>
<class 'Practical04_Support.Obstacle.Circle'>
<class 'Practical04_Support.Obstacle.Circle'>
<class 'Practical04_Support.Obstacle.Circle'>


HTML(value='\n                <div style="height: 500px; width: 100%; overflow-x: auto; overflow-y: hidden; re…